In [2]:
import os
import pandas as pd

In [4]:
csv_path = '../raw_data'
cleandf = pd.read_csv(os.path.join(csv_path, 'clean_data.csv'))

In [5]:
cleandf

,Unnamed: 0,text,target
0,0,smell hillary fear daniel greenfield shillman ...,0
1,1,watch exact moment paul ryan commit political ...,0
2,2,kerry go paris gesture sympathy u secretary st...,1
3,3,bernie supporter twitter erupt anger dnc try w...,0
4,4,battle new york primary matter primary day new...,1
...,...,...,...
6330,6330,state department say can find email clinton sp...,1
6331,6331,p pb stand plutocratic pentagon p pb stand plu...,0
6332,6332,anti trump protester tool oligarchy informatio...,0
6333,6333,ethiopia obama seek progress peace security ea...,1


In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Embedding, Flatten, Masking, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

2022-08-22 17:10:54.083882: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-08-22 17:10:54.168860: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-22 17:10:54.168879: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
tokenizer = Tokenizer()
model = Sequential()
es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
X_train, X_test, y_train, y_test = train_test_split(cleandf['text'], cleandf['target'], test_size=0.2)

2022-08-22 17:10:57.941092: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-22 17:10:57.941139: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-22 17:10:57.941161: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-0UJ0SON8): /proc/driver/nvidia/version does not exist
2022-08-22 17:10:57.942075: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
tokenizer.fit_on_texts(X_train)

In [8]:
X_train_token = tokenizer.texts_to_sequences(X_train)
X_test_token = tokenizer.texts_to_sequences(X_test)

In [9]:
X_train_token = pad_sequences(X_train_token, dtype='int32', padding='post', maxlen=300)
X_test_token = pad_sequences(X_test_token, dtype='int32', padding='post', maxlen=300)

In [10]:
model.add(Embedding(
    input_dim=(len(tokenizer.word_index))+1, # +1 for the 0 padding
    input_length=300, # Max_sentence_length (optional, for model summary)
    output_dim=100,
    mask_zero=True, # Built-in masking layer :)
))
#model.add(GRU(20, return_sequences=True))
model.add(Conv1D(30, kernel_size=5, activation='tanh'))
model.add(Flatten())
model.add(Dense(15, activation='relu'))
#model.add(Dropout(rate=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
              optimizer='rmsprop',
             metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 100)          4960900   
                                                                 
 conv1d (Conv1D)             (None, 296, 30)           15030     
                                                                 
 flatten (Flatten)           (None, 8880)              0         
                                                                 
 dense (Dense)               (None, 15)                133215    
                                                                 
 dense_1 (Dense)             (None, 1)                 16        
                                                                 
Total params: 5,109,161
Trainable params: 5,109,161
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(X_train_token, y_train, 
          epochs=50, 
          batch_size=32,
          validation_split=0.2,
          callbacks=[es],
        use_multiprocessing=True
         )

Epoch 1/50
127/127 [==============================] - 3s 17ms/step - loss: 0.4185 - accuracy: 0.7955 - val_loss: 0.2522 - val_accuracy: 0.8955
Epoch 2/50
127/127 [==============================] - 2s 16ms/step - loss: 0.1253 - accuracy: 0.9521 - val_loss: 0.3311 - val_accuracy: 0.8876
Epoch 3/50
127/127 [==============================] - 2s 16ms/step - loss: 0.0292 - accuracy: 0.9906 - val_loss: 0.3220 - val_accuracy: 0.9004
Epoch 4/50
127/127 [==============================] - 2s 19ms/step - loss: 0.0089 - accuracy: 0.9970 - val_loss: 0.4599 - val_accuracy: 0.8994
Epoch 5/50
127/127 [==============================] - 2s 17ms/step - loss: 0.0029 - accuracy: 0.9990 - val_loss: 0.5601 - val_accuracy: 0.8876
Epoch 6/50
127/127 [==============================] - 2s 16ms/step - loss: 0.0012 - accuracy: 0.9998 - val_loss: 0.6777 - val_accuracy: 0.8757
Epoch 7/50
127/127 [==============================] - 2s 18ms/step - loss: 4.5255e-05 - accuracy: 1.0000 - val_loss: 0.7619 - val_accuracy: 0.

In [13]:
model.evaluate(X_test_token, y_test, verbose=1)

40/40 [==============================] - 0s 5ms/step - loss: 0.2481 - accuracy: 0.8958


[0.24811822175979614, 0.8958168625831604]

# Attention: Beggining of Word2Vec
## Better run only on Google Colab

In [8]:
from gensim.models import Word2Vec
import numpy as np
import gensim.downloader as api

In [9]:
#word2vec = api.load('glove-wiki-gigaword-50') # this one crashes my kernel everytime
word2vec = api.load('text8') # this one takes an eternity

[==================================================] 100.0% 31.6/31.6MB downloaded


In [10]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

In [11]:
# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

In [12]:
# Embed the training and test sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)

KeyboardInterrupt: 

In [ ]:
X_train_embed = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=300)
X_test_embed = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=300)

In [ ]:
for X in [X_train_embed, X_test_embed]:
    assert type(X) == np.ndarray
    assert X.shape[-1] == word2vec.vector_size


assert X_train_embed.shape[0] == len(X_train)
assert X_test_embed.shape[0] == len(X_test)

In [ ]:
model = Sequential()
model.add(Masking(mask_value=0, input_shape=X_train_embed.shape[1:]))
#model.add(LSTM(20, return_sequences=True))
model.add(Conv1D(30, kernel_size=5, activation='tanh'))
model.add(Flatten())
model.add(Dense(15, activation='relu'))
#model.add(Dropout(rate=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
              optimizer='rmsprop',
             metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(X_train_embed, y_train, 
          epochs=50, 
          batch_size=32,
          validation_split=0.2,
          callbacks=[es],
        use_multiprocessing=True
         )

In [ ]:
model.evaluate(X_test_embed, y_test, verbose=1)